In [72]:
import numpy as np
import csv
%matplotlib inline
import pandas as pd
import re

from datetime import datetime

In [201]:
id_to_region = { 12: 1, 88: 1, 87: 1, 209: 1, 45: 1, 231: 1, 261: 1, 13: 1, 158: 2, 249: 2, 113: 2, 114: 2, 79: 2, 4: 2, 232: 2, 148: 2, 144: 2, 211: 2, 125: 2, 246: 3, 50: 3, 48: 3, 68: 3, 90: 3, 186: 3, 100: 3, 230: 3, 163: 3, 161: 3, 164: 3, 234: 3, 107: 3, 170: 3, 162: 3, 229: 3, 233: 3, 137: 3, 224: 3, 143: 4, 142: 4, 239: 4, 238: 4, 151: 4, 24: 4, 75: 5, 236: 5, 263: 5, 262: 5, 140: 5, 141: 5, 237: 5, 166: 6, 41: 6, 74: 6, 42: 6, 152: 6, 116: 6, 244: 7, 120: 7, 243: 7, 127: 7, 128: 7,
}
valid_ids = id_to_region.keys()
columns = ['weekday', 'hour', 'region', 'count', 'total_amount_sum', 'duration_min', 'duration_sum', 'duration_max']

In [229]:
def filter_pickup(data_pickup):
    # Filter all trips that start and end in the above regions
    # Create a copy
    data_pickup_filtered = chunk.copy()

    # Initialize the list to record which region the origin is located
    in_which_region_list = []

    # Loop through each row
    for i in range(0, len(data_pickup)):
        in_which_region = -1 # Initialize with -1
        if data_pickup["PULocationID"][i] in valid_ids and data_pickup["PULocationID"][i] == data_pickup["DOLocationID"][i]:
            in_which_region = id_to_region[data_pickup["PULocationID"][i]]
        in_which_region_list.append(in_which_region)
    data_pickup_filtered['region'] = in_which_region_list
    # Keep only those have real region indice
    data_pickup_filtered = data_pickup_filtered[data_pickup_filtered.region != -1] 
    # Reset the indice
    data_pickup_filtered = data_pickup_filtered.dropna(how='any').reset_index(drop=True)
    return data_pickup_filtered


def process_datetime(data_datetime):     
    weekday_list = []
    hour_list = []
    duration_list = []
    count = [1] * len(data_datetime)
    
    for i in range(len(data_datetime)):
        start = pd.to_datetime(data_datetime["tpep_pickup_datetime"][i])
        end = pd.to_datetime(data_datetime["tpep_dropoff_datetime"][i])
        weekday_list.append(start.weekday())
        hour_list.append(start.hour)
        duration_list.append(int((end-start).total_seconds()))

    data_datetime["weekday"] = weekday_list
    data_datetime["hour"] = hour_list
    data_datetime["duration"] = duration_list
    data_datetime["count"] = count
    return data_datetime


def compute_average(to_compute):
    df = to_compute.copy()
    # Remove unuseful columns
    df = df.drop(
        [
            "PULocationID", 
            "DOLocationID", 
            "tpep_pickup_datetime", 
            "tpep_dropoff_datetime", 
            "VendorID",
            "passenger_count",
            "RatecodeID",
            "store_and_fwd_flag",
            "payment_type",
            "fare_amount",
            "extra",
            "mta_tax",
            "tip_amount",
            "tolls_amount",
            "improvement_surcharge",
        ],
        1
    )
    # groupby
    df = df.groupby(["weekday","hour","region"], as_index=False).agg(
        {"count": "count", "total_amount": "sum", "duration": ["min", "sum", "max"]}
    )
    
    # rename the columns
    df.columns = columns
    return df


def process_pipeline(df, chunk, i):
    chunk = filter_pickup(chunk)
    chunk = process_datetime(chunk)
    chunk = compute_average(chunk)
    # combine chunk to df 

    if i == 0:
        df = chunk
    else:
        df = pd.concat([df, chunk], ignore_index=True)
        df = df.groupby(["weekday","hour","region"], as_index=False).agg(
            {
                "count": "sum", 
                "total_amount_sum": "sum", 
                "duration_min": "min", 
                "duration_sum": "sum", 
                "duration_max": "max",
            }
        )
    return df

In [230]:
chunksize = 10 ** 5
filename = "data/2018_Yellow_Taxi_Trip_Data.csv"
output = "data/2018_Taxi_Processed.csv"
current_size = 0
i = 0
df = pd.DataFrame(columns=columns)

for chunk in pd.read_csv(filename, chunksize=chunksize):
    current_size += len(chunk)
    # reset indices
    chunk = chunk.dropna(how='any').reset_index(drop=True)
    print("Iterations run: %s, rows read: %s" % (i, current_size))
    df = process_pipeline(df, chunk, i)
    i += 1

# save to file
print("saving to csv file %s" % output)
df.to_csv(output)

Iterations run: 0, rows read: 10000
[3 2]
   weekday hour region count total_amount duration             
                       count          sum      min    sum   max
0        2   22      5     1         6.96      232    232   232
1        3    5      3     1         3.30       10     10    10
2        3    5      4     1         4.80       92     92    92
3        3    5      6     1         3.80       71     71    71
4        3    6      1     3        19.60      128    678   330
5        3    6      2     2        76.03       11    171   160
6        3    6      3    33       297.43        3   6514   424
7        3    6      4     5       137.06        5    336   175
8        3    6      5    29       231.21       35   5480   497
9        3    6      6     6        34.50       93   1479   475
10       3    7      1    30       241.50       78  10944  1414
11       3    7      2    27       196.75       28   8685  1656
12       3    7      3   143      1176.28        5  37282  263

In [249]:
output = ("data/2018_Taxi_Aggregated.csv")
df = pd.read_csv("data/2018_Taxi_Processed.csv")

"""
We need to calculate the following:

Min, Average, Max number of trips (Cmin, C, Cmax) per hour per region
Average duration of trip (D) per hour per region
Average Fare of trip (F) per hour per region
""" 
df["min_trips"] = 3600/(df.duration_sum/df["count"]+180)


0     4.897413
1     5.394635
2     6.112486
3     9.600480
4     8.922196
5     6.549528
6     9.650749
7     3.867524
8     3.794689
9     6.538640
10    5.940224
11    9.144683
12    4.422345
13    8.063140
14    2.763539
15    6.933509
16    5.487311
17    9.333741
18    9.530081
19    5.141572
20    9.100393
21    1.964436
22    8.832351
23    5.469933
24    9.317894
25    4.096976
26    8.758488
27    9.398309
28    6.895351
29    3.885025
        ...   
70    6.559789
71    5.432104
72    5.783963
73    8.308783
74    7.332521
75    4.268996
76    7.925050
77    4.211822
78    6.758035
79    5.858837
80    7.604830
81    6.215149
82    8.693971
83    6.998012
84    5.372079
85    4.898141
86    6.374402
87    6.447969
88    6.670066
89    6.588048
90    8.041536
91    7.260056
92    7.994424
93    5.630496
94    7.395386
95    7.102816
96    8.724888
97    8.122392
98    7.413807
99    7.463099
Name: min_trips, Length: 100, dtype: float64
